## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,53.26,77,51,0.00,broken clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,79.41,73,0,31.81,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,87.12,50,0,21.85,clear sky
3,3,Ambilobe,MG,-13.2000,49.0500,80.78,82,15,8.75,light rain
4,4,Busselton,AU,-33.6500,115.3333,56.08,77,65,15.21,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Port Alfred,ZA,-33.5906,26.8910,79.41,73,0,31.81,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,87.12,50,0,21.85,clear sky
3,3,Ambilobe,MG,-13.2000,49.0500,80.78,82,15,8.75,light rain
5,5,Labuan,MY,5.2767,115.2417,80.96,83,20,2.30,few clouds
13,13,Atuona,PF,-9.8000,-139.0333,78.31,75,15,20.51,few clouds
17,17,Hithadhoo,MV,-0.6000,73.0833,82.89,70,31,10.92,scattered clouds
23,23,Alyangula,AU,-13.8483,136.4192,80.91,74,100,10.42,overcast clouds
26,26,Kavaratti,IN,10.5669,72.6420,78.69,66,57,8.50,broken clouds
28,28,Bengkulu,ID,-3.8004,102.2655,78.21,81,99,8.03,overcast clouds
34,34,Bambous Virieux,MU,-20.3428,57.7575,82.65,78,75,11.50,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                214
City                   214
Country                210
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clear_cities_df = preferred_cities_df.dropna()
clear_cities_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clear_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Alfred,ZA,79.41,clear sky,-33.5906,26.8910,
2,Cape Town,ZA,87.12,clear sky,-33.9258,18.4232,
3,Ambilobe,MG,80.78,light rain,-13.2000,49.0500,
5,Labuan,MY,80.96,few clouds,5.2767,115.2417,
13,Atuona,PF,78.31,few clouds,-9.8000,-139.0333,
17,Hithadhoo,MV,82.89,scattered clouds,-0.6000,73.0833,
23,Alyangula,AU,80.91,overcast clouds,-13.8483,136.4192,
26,Kavaratti,IN,78.69,broken clouds,10.5669,72.6420,
28,Bengkulu,ID,78.21,overcast clouds,-3.8004,102.2655,
34,Bambous Virieux,MU,82.65,light rain,-20.3428,57.7575,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...Skipping!!")
        pass
        

Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!
Hotel not found...Skipping!!


In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Alfred,ZA,79.41,clear sky,-33.5906,26.8910,The Halyards Hotel
2,Cape Town,ZA,87.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
3,Ambilobe,MG,80.78,light rain,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
5,Labuan,MY,80.96,few clouds,5.2767,115.2417,Tiara Labuan Hotel
13,Atuona,PF,78.31,few clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
672,Itarema,BR,87.08,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
673,Calabozo,VE,86.81,broken clouds,8.9242,-67.4293,Hotel Giardini
682,San Fernando,PH,80.15,scattered clouds,15.0286,120.6898,Maharlika Hostel
683,Acarau,BR,86.41,scattered clouds,-2.8856,-40.1200,castelo pizzaria


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current Description}{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))